In [1]:
# convert to a '.py' file
#!jupyter nbconvert --to script *.ipynb

In [2]:
!pip3 install jams
!pip install matplotlib
!pip install audiolazy
!pip install librosa

In [1]:
import numpy as np
import os
import pprint

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

import scipy.io.wavfile as wav
import jams

import librosa
import audiolazy as al

## Process input

In [2]:
def timeKey(t):
    return t[0]

def loadNoteData(f):
    #f = open('/content/JamsFiles/04_Rock1-90-C_solo.jams', 'r')

   # data = f.read()
    data = jams.load(f)
    notes = []

    for i in range (0, 6):
        for j in data.annotations["note_midi"][i]["data"]:
            notes.append([j[0],j[1],j[2]])
            
    notes.sort(key=timeKey)

    #pprint.pprint(notes)
    return notes

In [3]:
def load_audio_file(song_path, scale = False):
    srate, source_audio = wav.read(song_path)
    if scale:
        source_audio = source_audio.astype(np.float32) / max(max(source_audio),abs(min(source_audio)))

    return source_audio, srate

In [4]:
#function to process the raw audio data.
def x_data_process(raw_data):
    scaler = preprocessing.MinMaxScaler((-1,1))
    scaled = scaler.fit_transform(raw_data)
    return scaled

In [5]:
#TODO: Change this so it pads to match the shape of frame. not just the length
def pad(proper_size, frame):
    padsize = winSize-len(frame)
    temp = None
    if len(frame.shape) > 1:
        temp = np.zeros((frame.shape[0],padsize))
    else:
        temp = np.zeros(padsize)

    return np.concatenate((frame,temp))

In [6]:
# iterate through each song, one frame at a time based off hop and window size
def songProcess(song_audio,notes,sr,hopSize = 1024,winSize = 4096,features = [],labels = []):

  testCount = 0

  offsets = np.arange(0,len(song_audio),hopSize)
  for (i,o) in enumerate(offsets):
    testCount += 1

    original_frame_size = 0

    frame = audio[o:o+winSize]

    if original_frame_size == 0:
      original_frame_size = frame.shape

    note = 0
    tiebreak = []

    #Create labels from jams file data -- if multiple notes in frame, take longest duration
    for j,(time,duration,value) in enumerate(notes):
      note_start = time*sr
      note_end = (time+duration)*sr
      if o <= note_start < o+winSize:                 #note starts in frame
        tiebreak.append(j)
      elif o <= note_end < o+winSize:                  #note ends in frame
        tiebreak.append(j)
      elif note_start < o and o+winSize <= note_end: #note continuous thru frame
        note = value

    #if multiple notes in frame, choose one that played the longest in frame
    if len(tiebreak) > 0:
      if len(tiebreak) == 1:
        note = notes[tiebreak[0]][2]
      else:
        max_dur = 0
        max_note = 0
        for index in tiebreak:
          note_start = notes[index][0] * sr
          note_duration = notes[index][1] * sr
          frame_dur = 0
          if note_start < o:
            frame_dur = note_duration+note_start-o
          elif note_start+note_duration > o+winSize:
            frame_dur = o+winSize - note_start
          else:
            frame_dur = note_duration

          if frame_dur > max_dur:
            max_dur = frame_dur
            max_note = notes[index][2]
        note = max_note


    #pad feature matrix
    if len(frame) < winSize:
      frame = pad(winSize, frame)

    #append to feature and labels
    features.append(frame)
    labels.append(round(note))  #quantize to the nearest midi value

  return features,labels,testCount

## Predict

In [7]:
"""Taking the song path, trained model, and same hopSize and winSize as used in training,
Converts song into predicted midi information"""
def predict(song_path, model, hopSize, winSize):
    
    audio,sr = load_audio_file(song_path, scale=False)
    offsets = np.arange(0,len(audio),hopSize)
    x_data = []
    for (i,o) in enumerate(offsets):
        frame = audio[o:o+winSize]
        #pad
        if len(frame) < winSize:
            frame = pad(winSize, frame)

        #append to features
        x_data.append(frame)

    #preprocess data
    x_data = x_data_process(x_data)

    #column 0 is note, column 1 is time
    midi_info = np.zeros((len(offsets),2))
    raw_results = model.predict(x_data)

    for i,frame_note in enumerate(raw_results):
        midi_info[i][0] = frame_note
        midi_info[i][1] = hopSize*i/sr

        #TODO: create proper midi file from this note / time information
          #Also determine method to covert midi into sheet music

    return midi_info

## Data classification using Neural Network

In [8]:
def neuralNetwork(XTrain, XTest, YTrain, YTest):
    #Currently, using successful values from A1, further adjustment with the processed dataset will be needed
    MLPC = MLPClassifier(hidden_layer_sizes=(71, 21), max_iter=10000, alpha=0.0001, learning_rate_init=0.001, solver='adam')
    MLPC.fit(XTrain, YTrain)

    return accuracy_score(YTest, MLPC.predict(XTest)), MLPC

In [9]:
def shawn_nn(XTrain, XTest, YTrain, YTest, hidden_size, max_iter, solver, activation):
    #Currently, using successful values from A1, further adjustment with the processed dataset will be needed
    MLPC = MLPClassifier(hidden_layer_sizes=hidden_size, max_iter=max_iter, alpha=0.0001, 
                         learning_rate_init=0.001, solver=solver, activation=activation)
    MLPC.fit(XTrain, YTrain)

    return accuracy_score(YTest, MLPC.predict(XTest)), MLPC


## Data classification using SVM

In [10]:
def gaussianSVM(XTrain, XTest, YTrain, YTest):
    #Currently, using successful values from A2, further adjustment with the processed dataset will be needed
    svc = SVC(kernel = 'rbf', gamma = 0.1, C = 10)
    svc.fit(XTrain, YTrain)

    return accuracy_score(YTest, svc.predict(XTest))

In [11]:
def linearSVM(XTrain, XTest, YTrain, YTest):
    #Currently, using successful values from A2, further adjustment with the processed dataset will be needed
    svc = LinearSVC(C = 0.0225)

    svc.fit(XTrain, YTrain)
    accuracy_score(YTest, svc.predict(XTest))

In [12]:
def linear_SVM(X_train, X_test, y_train, y_test, r_values):
    models = []
    train_scores = []
    test_scores = []
    
    for r_value in r_values:
        model = SVC(kernel='linear', C=r_value)
        model.fit(X_train, y_train)
        models.append(model)
        
    for model in models:
        train_predict = model.predict(X_train)
        train_error = hinge_loss(y_train, train_predict)
        train_scores.append(train_error)
    
        test_predict = model.predict(X_test)
        test_error = hinge_loss(y_test, test_predict)
        test_scores.append(test_error)
        
    # The following logic was reused from my assignment 1 code
    
    # Plotting the Error over my Regularization Value(s)
    fig, ax = plt.subplots()
    ax.set_xlabel("Regularization Value(s)")
    ax.set_ylabel("Error (Hinge)")
    ax.set_title("Error vs Regularization Values(s) for Training and Testing")
    ax.plot(r_values, train_scores, marker="o", label="Train", drawstyle="steps-post")
    ax.plot(r_values, test_scores, marker="o", label="Test", drawstyle="steps-post")
    ax.legend()
    plt.show()
    
    # For my analysis I want to find the best C(regularization) value 
    # for the logistic regression from my scenario and its error 
    best_train_r_index = train_scores.index(min(train_scores))
    best_train_r = r_values[best_train_r_index]
    best_train_error = train_scores[best_train_r_index]
    
    best_test_r_index = test_scores.index(min(test_scores))
    best_test_r = r_values[best_test_r_index]
    best_test_error = test_scores[best_test_r_index]
    
    # Printing what training and testing resulted in an error according to the r value used 
    print("Linear SVM Model\n")
    print("Regularization Values Used:\n",r_values)
    print("\nLowest Error for Training:", best_train_error)
    print("R Value Used:", best_train_r)
    print("\nLowest Error for Testing:", best_test_error)
    print("R Value Used:", best_test_r)

## Driver

In [13]:
# LOAD SONG AND JAM FILES
song_path = r'DataSets/audio_mono-mic'
jam_path = r'DataSets/annotation'
MODE = 'solo'
inputFiles = list(zip([x for x in os.listdir(song_path) if MODE in x],[x for x in os.listdir(jam_path) if MODE in x]))

In [14]:
##Set up features and labels for ML
numSongs = 15
counter = numSongs

notecounter = 0

hopSize = 1024
winSize = 1024

XData,YData = [],[]
for song_file,jam_file in inputFiles:
    song = os.path.join(song_path,song_file)
    jam = os.path.join(jam_path,jam_file)
    
    audio_prescaled,sr = load_audio_file(song,scale=True)
    audio,sr = load_audio_file(song, scale=False)
    note_info = loadNoteData(jam)

    #NOTE: window size used in previous works with dataset where 0.2s
    XData_prescaled,YData,notecount = songProcess(audio_prescaled,note_info,sr, hopSize=hopSize, winSize = winSize) # Can pass in a feature matrix and label array if we want to concat multiple songs together


    XData,YData,notecount = songProcess(audio,note_info,sr, hopSize=hopSize, winSize = winSize) # Can pass in a feature matrix and label array if we want to concat multiple songs together


    notecounter += notecount
    counter -= 1
    if counter == 0:
        break


In [15]:
XData_prescaled = np.array(XData_prescaled)
XData_prescaled = XData_prescaled / max(XData_prescaled.max(), abs(XData_prescaled.min()))
# print(X1.min(axis=0))
# print(X1.max(axis=0))

# XData_post = np.array(x_data_process(XData))
# print(X2.min(axis=0))
# print(X2.max(axis=0))

In [30]:
### XDATA SCALING EXPERIMENT
#Method 1: perform a -1,1 scale on the audio file at initial loading. This scales audio files individually, not recognizing the values of other files
#Method 2: perform a -1,1 scale at train test split via sklearn. This scaled audio files together, after they are loaded and concatenated.
#Method 3: control. No scaling
XData_prescaled = np.array(XData_prescaled)
XData_prescaled = XData_prescaled / max(XData_prescaled.max(), abs(XData_prescaled.min()))
XData_postscaled = x_data_process(XData)

N_avg = 3
results = {}
for j,xdata in enumerate((XData_prescaled,XData_postscaled,XData)):
    results[j] = 0
    XTrain,XTest,YTrain,YTest = train_test_split(xdata,YData,test_size=0.2)
    for i in range(N_avg):
        results[j] += neuralNetwork(XTrain,XTest,YTrain,YTest)[0]
    results[j] /= N_avg

print("Method 1 accuracy:",results[0])
print("Method 2 accuracy:",results[1])
print("Control. No scaling accuracy:",results[2])

Method 1 accuracy: 0.5550583129915921
Method 2 accuracy: 0.47721724979658253
Control. No scaling accuracy: 0.2615947925142392


## NN Training - Different Models

In [31]:
#Default Train model
XData_postscaled = x_data_process(XData)

XTrain,XTest,YTrain,YTest = train_test_split(XData_postscaled,YData,test_size=0.2)
accuracy, trained_model = neuralNetwork(XTrain,XTest,YTrain,YTest)
print("NN Model Test Accuracy:",accuracy)

NN Model Test Accuracy: 0.49918633034987797


In [33]:
# Shawn's Testing - best model so far - takes about 20min to run however
XData_postscaled = x_data_process(XData)
XTrain,XTest,YTrain,YTest = train_test_split(XData_prescaled,YData,test_size=0.2)
hidden_size=(80,120)
max_iter=100000
solver='lbfgs'
activation='tanh'
accuracy, trained_model = shawn_nn(XTrain,XTest,YTrain,YTest,hidden_size, max_iter, solver, activation)
print("NN Model Test Accuracy:",accuracy)


NN Model Test Accuracy: 0.8234336859235151


/Users/shawnnettleton/opt/miniconda3/envs/apress/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [45]:
XData_postscaled = x_data_process(XData)
XTrain,XTest,YTrain,YTest = train_test_split(XData_prescaled,YData,test_size=0.2)
hidden_size=(80,120)
max_iter=10000
solver='adam'
activation='tanh'
accuracy, trained_model = shawn_nn(XTrain,XTest,YTrain,YTest,hidden_size, max_iter, solver, activation)
print("NN Model Test Accuracy:",accuracy)

NN Model Test Accuracy: 0.7563059397884458


In [16]:
# Best of both - fast acting and good accuracy
XData_postscaled = x_data_process(XData)
XTrain,XTest,YTrain,YTest = train_test_split(XData_prescaled,YData,test_size=0.2)
hidden_size=(60,120)
max_iter=10000
solver='adam'
activation='tanh'
accuracy, trained_model = shawn_nn(XTrain,XTest,YTrain,YTest,hidden_size, max_iter, solver, activation)
print("NN Model Test Accuracy:",accuracy)

NN Model Test Accuracy: 0.5946780358718025


## SVM Training - Different Models

In [ ]:
### XDATA SCALING EXPERIMENT
#Method 1: perform a -1,1 scale on the audio file at initial loading. This scales audio files individually, not recognizing the values of other files
#Method 2: perform a -1,1 scale at train test split via sklearn. This scaled audio files together, after they are loaded and concatenated.
#Method 3: control. No scaling
XData_prescaled = np.array(XData_prescaled)
XData_prescaled = XData_prescaled / max(XData_prescaled.max(), abs(XData_prescaled.min()))
XData_postscaled = x_data_process(XData)

c0 = 0.05
alpha = 2.3
r_values = []
for alpha_exponent in range(10):
    r_value = (alpha ** alpha_exponent) * c0
    r_values.append(r_value)

N_avg = 3
results = {}
for j,xdata in enumerate((XData_prescaled,XData_postscaled,XData)):
    results[j] = 0
    XTrain,XTest,YTrain,YTest = train_test_split(xdata,YData,test_size=0.2)
    for i in range(N_avg):
        results[j] += linear_SVM(XTrain,XTest,YTrain,YTest,r_values)[0]
    results[j] /= N_avg

print("Method 1 accuracy:",results[0])
print("Method 2 accuracy:",results[1])
print("Control. No scaling accuracy:",results[2])

In [ ]:
#Default Train model
XData_postscaled = x_data_process(XData)

c0 = 0.05
alpha = 2.3
r_values = []
for alpha_exponent in range(10):
    r_value = (alpha ** alpha_exponent) * c0
    r_values.append(r_value)

XTrain,XTest,YTrain,YTest = train_test_split(XData_postscaled,YData,test_size=0.2)
accuracy, trained_model = linear_SVM(XTrain,XTest,YTrain,YTest,r_values)
print("NN Model Test Accuracy:",accuracy)

In [103]:
## Call predict function
song = 'DataSets/audio_mono-mic/00_BN3-119-G_solo_mic.wav'

midi_info = predict(song, trained_model, hopSize, winSize)

# print(midi_info[:10])

[[0.         0.        ]
 [0.         0.02321995]
 [0.         0.04643991]
 [0.         0.06965986]
 [0.         0.09287982]
 [0.         0.11609977]
 [0.         0.13931973]
 [0.         0.16253968]
 [0.         0.18575964]
 [0.         0.20897959]]


## Testing Ground

In [13]:
f = open('/content/JamsFiles/04_Rock1-90-C_solo.jams', 'r')

# data = f.read()
data = jams.load(f)

In [ ]:
print("\ttime\tduration\tnotes")
pprint.pprint(loadNoteData(data))

In [96]:
#File loading Test
for song,jam in inputFiles:
    if song[:-13] != jam[:-10]:
        print("error with:",song,jam)

In [ ]:
#Test XData padding
for x in XData:
    if len(x) < 4096:
        print("Error with Xdata padding")

In [137]:
#Check to see if XData and YData arrays are being appended properly
if notecounter != len(XData):
    print("Error: XData and YData arrays are not being apppended properly")

In [205]:
# From the predict() function above to get the note and time info we can see what note and freq it represents

# Reference: https://pythonhosted.org/audiolazy/lazy_midi.html

print('MIDI \tNote  \tFreq \t\t\tTime')
for note, time in midi_info:
    if note != 0.0:
        print(note,'\t',al.midi2str(note),'\t',al.midi2freq(note),'\t',time)

MIDI 	Note  	Freq 			Time
50.0 	 D3 	 146.8323839587038 	 1.8575963718820863
60.0 	 C4 	 261.6255653005986 	 1.9504761904761905
60.0 	 C4 	 261.6255653005986 	 3.9706122448979593
50.0 	 D3 	 146.8323839587038 	 5.828208616780046
60.0 	 C4 	 261.6255653005986 	 7.894784580498866
60.0 	 C4 	 261.6255653005986 	 20.456780045351476
60.0 	 C4 	 261.6255653005986 	 21.91963718820862
69.0 	 A4 	 440.0 	 22.709115646258503
